In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

import sys
sys.path.append('C:/Users/matsz/programowanie/Optymalizacja portfela/esg')
import preproccessing as prep
from regime_switching_brownian_motion import RegimeSwitching
from independent_lognormal_model import IndependentLogNormal


### TODO
refactor 

In [2]:
""" spec = [
    ('initial_value', numba.float64[:]),
    ('inflows', numba.float64[:]),
    ('returns', numba.float64[:,:,:]),
    ('capital', numba.float64[:])
]
@jitclass(spec)  """
class Simulation():
    def __init__(self, initial_value, inflows, returns) -> None:
        self.initial_value = np.full(returns.shape[1], initial_value)
        self.returns = returns
        self.inflows = inflows

    def step(self, current_value,inflow,weights,returns):
        portfolio_return = returns @ weights
        next_assets_value =  (current_value + inflow) * (1+portfolio_return)
        return next_assets_value

    def run(self, strategy):
        self.capital = self.initial_value
        T = len(strategy)
            
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],strategy[t],self.returns[t])


class MaxPercentile():

    def __init__(self, percentile) -> None:
        self.percentile = percentile

    def __call__(self, values: np.array) -> float:
        #return np.round(np.percentile(values,self.percentile))
        return np.percentile(values,self.percentile)

class Fitness:
    def __init__(self,simulator: Simulation,portfolios,objective) -> None:
        self.simulator = simulator
        self.portfolios = portfolios
        self.objective = objective

    def __call__(self, solution: np.array) :
        strategy = np.take(self.portfolios,solution, axis=0)
        self.simulator.run(strategy)
        result = self.objective(self.simulator.capital)
        return result
    
def glide_path_grid_search(T, capital, inflows,risk, portfolios, returns):
    l = np.arange(T,0,-1)
    capital = np.repeat(capital,returns.shape[1])

    glide_paths = np.full(T+1,len(portfolios))
    perc = np.zeros((l.shape[0],len(portfolios)))

    objective = MaxPercentile(risk)
    simulator = Simulation(capital,inflows,returns)
    fitness = Fitness(simulator,portfolios,objective)

    for t in l:

        for portfolio in range(len(portfolios)):

            solution = np.full(t,portfolio)
            
            perc[t-1,portfolio] = fitness(solution)
        glide_paths[t-1] = np.argmax(perc[t-1])
    
    return np.flip(glide_paths)[1:]

#optimize glide with resamping
def resampled_glide_path_grid_search(T,capital, inflows, returns, risk, portfolios, number_of_runs):
    batches = np.asarray(np.array_split(returns,number_of_runs,axis=1))
    glides = np.zeros((number_of_runs,T))


    for batch in range(len(batches)):
        glide = glide_path_grid_search(T,capital,inflows,risk,portfolios,batches[batch])
        glides[batch] = glide

    glides = np.int32(glides)

    result = np.zeros(T)
    for t in range(T):
        result[t] = np.argmax(np.bincount(glides[:,t], minlength=11))
        
    return pd.Series(result, index=range(T,0,-1))

In [3]:
def expected_returns(period):
    return pd.Series({
		'ACWI': 0.07/period,
		'^TBSP': 0.035/period,
		'XAUPLN': 0.05/period,
		'MWIG40': 0.065/period,
		'IHYU.L': 0.038/period,
		'PLOPLN3M': 0.03/period,
		'edo': 0.025/period
})
#acwi and edo
corr = np.array([[1,0],[0,1]])
std = np.array([0.135,0.02])
cov_matrix = prep.corr_to_cov(corr, std)

assets = ['ACWI', 'edo']
means = expected_returns(1)[assets]
number_of_scenarios = 10 * 10000
T = 50
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov_matrix, T))

#calculate inflation
sim_returns = np.expm1(sim_returns)
sim_returns[:,:,1] = np.where(sim_returns[:,:,1]<0,0,sim_returns[:,:,1])
sim_returns[:,:,1] = sim_returns[:,:,1] + 0.0123

sim_returns = np.array(sim_returns).transpose(1,0,2)
sim_returns.shape

(50, 100000, 2)

In [5]:
1- 0.108

0.892

In [19]:
model = RegimeSwitching(models = [IndependentLogNormal(mu=0.0127,sigma=0.027),
              IndependentLogNormal(mu=-0.0136, sigma=0.0510)],
    probs = [[0.958,0.042],[0.108, 0.892]]
)
equity = model.scenarios(100, current_regimes=0, dt=1,n_scenarios=10000,n_steps=300)
equity = equity.T
equity_returns = equity[12::12]/equity[:-12:12]-1
equity_returns.shape

(25, 10000)

In [21]:
equity[:13,0]
print(115.28581515/100-1)
equity_returns[:12,0]

0.15285815150000004


array([ 0.17494115,  0.09203997, -0.12143648,  0.00331105,  0.11267834,
        0.11149622,  0.21439745,  0.07967982,  0.18423988, -0.19330411,
       -0.01171777,  0.1969938 ])

In [43]:
""" portfolios = [[0.0,1.0],
             [0.2,0.8],
             [0.4,0.6],
             [0.6,0.4],
             [0.8,0.2],
             [1.0,0.0]]  """

portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,0.0]]



In [45]:
number_of_runs = 10
T = 50
capital = 10.0
#risks = [5,15,25,35,45]
#risks = [10,20,30,40,50]
risks = [5,10,15,20,25,30,35,40,45]
inflows = np.cumproduct(np.full(T,1.025))*5

glide_paths = np.zeros((T,len(risks)))

for risk in range(len(risks)):
    glide_paths[:,risk] = resampled_glide_path_grid_search(T,capital,inflows,sim_returns,risks[risk],portfolios,number_of_runs)

glide_paths = pd.DataFrame(data=np.int32(glide_paths),index=range(T,0,-1), columns=risks)
glide_paths

,5,10,15,20,25,30,35,40,45
50,5,9,10,10,10,10,10,10,10
49,5,10,10,10,10,10,10,10,10
48,4,9,10,10,10,10,10,10,10
47,4,9,10,10,10,10,10,10,10
46,4,9,10,10,10,10,10,10,10
45,4,9,10,10,10,10,10,10,10
44,4,8,10,10,10,10,10,10,10
43,4,8,10,10,10,10,10,10,10
42,4,8,10,10,10,10,10,10,10
41,3,8,10,10,10,10,10,10,10


In [12]:
glide_paths.to_excel('glide_paths_norm_5.11.2023.xlsx')

In [13]:
#TODO sim returns for glide paths

inflows = np.cumproduct(np.full(T,1.025))*0
initial_capital = 1.0
simulator = Simulation(initial_capital,inflows,sim_returns)

glide_returns = np.zeros((T,len(risks)))

for t in range(0,T,):
    for risk in range(len(risks)):
        glide_path = glide_paths.loc[t+1:,risks[risk]]
        fitness = Fitness(simulator,portfolios,MaxPercentile(risks[risk])) 
        glide_returns[t,risk] = fitness(glide_path)         
      
glide_returns = pd.DataFrame(data=np.flip(glide_returns,axis=0),index=range(T,0,-1),columns=risks)
glide_returns

,5,10,15,20,25,30,35,40,45
50,6.546945,7.508613,9.374345,12.015180,15.012912,18.295282,21.969882,25.409959,29.383905
49,6.281831,7.160237,8.868244,11.309276,14.082984,17.147384,20.552440,23.736555,27.381901
48,6.027142,6.846217,8.385197,10.645511,13.230783,16.077842,19.252323,22.172214,25.573536
47,5.790037,6.523636,7.924819,10.024153,12.427097,15.080393,18.010812,20.739884,23.857685
46,5.571239,6.220114,7.501779,9.446612,11.673024,14.142562,16.849018,19.375876,22.280162
45,5.340643,5.945544,7.101745,8.907982,10.968209,13.250789,15.783249,18.115702,20.810908
44,5.122674,5.684635,6.729721,8.400522,10.311219,12.425319,14.771963,16.951846,19.396021
43,4.917669,5.435014,6.372570,7.914130,9.690438,11.637325,13.835999,15.859563,18.136040
42,4.721133,5.194796,6.033460,7.462427,9.111297,10.907520,12.950737,14.820225,16.923200
41,4.538187,4.968344,5.719264,7.037803,8.560642,10.235257,12.123878,13.853935,15.792940


In [14]:
description = pd.DataFrame(data={'Opis': """"
Numer_of_scenarios = 250000,
number_of_runs = 100
T = 50
capital = 10.0
risks = [5,10,15,20,25,30,35,40,45]
inflows = np.cumproduct(np.full(T,1.025))*5
"""},index=[1])

In [16]:
with pd.ExcelWriter('glide_paths_norm_5.11.2023.xlsx', engine='openpyxl', mode='w') as writer:  
    glide_paths.to_excel(writer, sheet_name='Glide_paths')
    glide_returns.to_excel(writer, sheet_name='Returns')
    description.to_excel(writer,sheet_name='Description')
